In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h1> Import Libraries Here </h1>


In [ ]:
import pandas as pd
import numpy as np
import json
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from gensim.models import Word2Vec
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
import copy


<h1> Read Training Data Here </h1>

In [ ]:
prepared_dataset=pd.read_csv('/content/drive/MyDrive/Math23K_English_with_data_prep.csv')
#prepared_dataset.head()

In [ ]:
with open('/content/drive/MyDrive/equations_data.json') as f:
  prefix_equations_list = json.load(f)
#print(len(prefix_equations_list))
#print(prefix_equations_list[0])

<h1> Extract Parts Of Prepared Dataset Here

In [ ]:
prepared_questions=prepared_dataset['prepared_question'].values.tolist()
#print(len(prepared_questions))
#print(prepared_questions[0])   

In [ ]:
normal_questions=prepared_dataset['segmented_question'].values.tolist()
#print(len(normal_questions))
#print(normal_questions[0])

In [ ]:
infix_equations=prepared_dataset['equation'].values.tolist()
#print(len(infix_equations))
#print(infix_equations[0])

In [ ]:
answers=prepared_dataset['ans'].values.tolist()
#print(len(answers))
#print(answers[0])

<h1> Preprocess The Data

<h2> Remove the unnecessary Data (No Prefix) </h2> 

In [ ]:
idx_to_remove=[i for i in range(len(prefix_equations_list)) if prefix_equations_list[i]=="no_prefix"]
prepared_questions=[prepared_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
normal_questions=[normal_questions[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
infix_equations=[infix_equations[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
answers=[answers[i] for i in range(len(prefix_equations_list)) if prefix_equations_list[i] != "no_prefix"]
prefix_equations_list=[equation for equation in prefix_equations_list if equation != "no_prefix"]


idx_to_remove_chinnese=[i for i in range(len(infix_equations)) if infix_equations[i]=="x=80千米/小时"]
print(idx_to_remove_chinnese)
if len(idx_to_remove_chinnese) >=1:
  prepared_questions.pop(idx_to_remove_chinnese[0])
  normal_questions.pop(idx_to_remove_chinnese[0])
  infix_equations.pop(idx_to_remove_chinnese[0])
  answers.pop(idx_to_remove_chinnese[0])
  prefix_equations_list.pop(idx_to_remove_chinnese[0])

# print(len(idx_to_remove))
# print(idx_to_remove)
# print(len(prefix_equations_list))
# print(len(prepared_questions))
# print(len(normal_questions)) 
# print(len(infix_equations))
# print(len(answers))

[10149]


In [ ]:
# print(prefix_equations_list[2578])
# print(prepared_questions[2578])
# print(normal_questions[2578])
# print(infix_equations[2578])
# print(answers[2578])

<h2> Tokenize  </h2>

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
temp_questions=list()
for elem in prepared_questions:
  temp_questions.append(word_tokenize(elem))
prepared_questions=temp_questions
#print(len(prepared_questions))
# print(prepared_questions[0])
# print(prepared_questions[0].index('n0'))
# print(prepared_questions[0].index('n1'))

<h1> Making Dict of Indexes of Unknowns (n's)

In [ ]:
unknown_dict_structure={'n0':-1,'n1':-1,'n2':-1,'n3':-1, 'n4': -1,'n5': -1,'n6': -1,'n7': -1,'n8': -1,'n9': -1,'n10': -1,'n11': -1,'n13': -1,'n15': -1}
unknowns_dict_list=[unknown_dict_structure.copy() for elem in prepared_questions]
unknowns_lst=unknown_dict_structure.keys()
# print(len(unknowns_lst))
unknowns_lst=list(unknowns_lst)
print(unknowns_lst)
# print(len(unknowns_dict_list))
# print(len(prepared_questions))

['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'n10', 'n11', 'n13', 'n15']


In [ ]:
for q in range(len(prepared_questions)):
  for w in range(len(prepared_questions[q])):
    # print(prepared_questions[q][w])
    # harsh_exit()
    # break
    if prepared_questions[q][w] in unknowns_lst:
      # print(prepared_questions[q][w])
      # print(w)
      # harsh_exit()
      #print(prepared_questions[q][w])

      unknowns_dict_list[q][prepared_questions[q][w]]=w
  


In [ ]:
# print(unknowns_dict_list[0])
# print(unknowns_dict_list[1])
# print(unknowns_dict_list[2])

# print(prepared_questions[1])
# # print(prepared_questions[2])


<h1> Create Vocab Input and Output Both</h1>

In [ ]:
index_to_word_input=['<pad>']
for question in prepared_questions:
  for word in question:
    if(word not in index_to_word_input):
      index_to_word_input.append(word)
#print(len(index_to_word_input))
#print(index_to_word_input[0])
index_to_word_input.append('<start>')
index_to_word_input.append('<end>')
input_vocab_size=len(index_to_word_input)
#input_vocab_size

In [ ]:
word_to_index_input={'<pad>':0}
for word in index_to_word_input:
  if word not in word_to_index_input.keys():
    word_to_index_input[word]=index_to_word_input.index(word)
# print(len(word_to_index_input))
# print(word_to_index_input["n0"])
# print(word_to_index_input["n1"])
# print(word_to_index_input["children"])
# print(word_to_index_input['<pad>'])

In [ ]:
index_to_word_output=list()
for equation in prefix_equations_list:
  for elem in equation:
    if elem not in index_to_word_output:
      index_to_word_output.append(elem)
# print(len(index_to_word_output))
# print(index_to_word_output[0])
index_to_word_output.append('<start>')
index_to_word_output.append('<end>')
output_vocab_size=len(index_to_word_output)
# print(output_vocab_size)
# print(index_to_word_output)

In [ ]:
word_to_index_output=dict()
for word in index_to_word_output:
  if word not in word_to_index_output.keys():
    word_to_index_output[word]=index_to_word_output.index(word)
# print(len(word_to_index_output))
# print(word_to_index_output)

<h1> Encode Data </h1>

In [ ]:
prepared_questions_temp=list()
for question in prepared_questions:
  question_temp=list()
  for word in question:
    question_temp.append(word_to_index_input[word])
  prepared_questions_temp.append(question_temp)
# print(prepared_questions[11235])
prepared_questions=prepared_questions_temp
# print(prepared_questions[11235])

In [ ]:
print(prepared_questions[0])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 10, 4, 11, 12, 13, 9, 14, 15, 16, 1, 3, 17, 11, 18, 19, 20, 21, 22, 15, 23, 17, 24, 25, 26, 4, 1, 13, 27, 28, 29, 30, 31, 11, 32, 4, 33, 15, 23, 17, 16, 34, 35, 21, 36, 37, 38, 39]


<h1> Padding </h1>

In [ ]:
max_len=max(len(question) for question in prepared_questions)
#print(max_len)
prepared_questions= sequence.pad_sequences(prepared_questions, maxlen=max_len,padding="post")
# print(prepared_questions[2])
# print(len(prepared_questions))
#reviews.shape
# print(prepared_questions[0])
# print(prepared_questions[0].index(20))
# print(prepared_questions[0].index(33))

In [ ]:
# print(word_to_index_input['n0'])
# print(word_to_index_input['n1'])
# print(torch.tensor(prepared_questions).shape)

<h1> Import Pytorch Dependencies Here

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader 
import torch.nn.functional as F
from torch.utils.data.sampler import WeightedRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau

<h1> Split Data into Train and Validation and Test</h1>

In [ ]:
prepared_questions_train, prepared_questions_test, prefix_equations_list_train, prefix_equations_list_test = train_test_split(prepared_questions,prefix_equations_list, train_size=0.9)
# print(len(prepared_questions_train))
# print(len(prefix_equations_list_train))
# print(len(prepared_questions_test))
# print(len(prefix_equations_list_test))

In [ ]:
prepared_questions_train, prepared_questions_val, prefix_equations_list_train, prefix_equations_list_val = train_test_split(prepared_questions_train,prefix_equations_list_train, train_size=0.9)

In [ ]:
# print(len(prepared_questions_train))
# print(len(prefix_equations_list_train))
# print(len(prepared_questions_val))
# print(len(prefix_equations_list_val))

<h1> Encoder </h1>

In [ ]:
class Encoder_GT(torch.nn.Module):
  def __init__(self,input_vocab_size,embedding_dim,hidden_dim):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.embedding=nn.Embedding(input_vocab_size+1,embedding_dim,padding_idx=0)
    self.bigru=nn.GRU(embedding_dim,hidden_dim,batch_first=True,bidirectional=True)
    self.dropout=nn.Dropout()
  
  def forward(self,question):
    h0=torch.zeros(1,question.size(0),self.hidden_dim)
    embedded_question=self.embedding(question)
    #print(embedded_question)
    embedded_question_dropout=self.dropout(embedded_question)
    #print(embedded_question_dropout)
    gru_rep,(hidden1,hidden2)=self.bigru(embedded_question_dropout)
    #print(gru_rep)
    gru_rep=gru_rep[:, :, :self.hidden_dim] +gru_rep[:, :, self.hidden_dim:]
    #print("-------------------------------")
    #print(gru_rep)
    #print(hidden1)
    #print(hidden2)
    return hidden1+hidden2,gru_rep


In [ ]:
# encoder_model=Encoder_GT(input_vocab_size,5,4)
# question=[[1,25,3],[25,2,45]]
# unknowns=[{'n0':1,'n1':-1,'n2':-1}],{'n0':0,'n1':2,'n2':-1}
# goal,gru_rep=encoder_model(torch.tensor(question))
# print(gru_rep)
# print(gru_rep[0][1])

In [ ]:
word_to_index_output

{'*': 0,
 '+': 5,
 '-': 1,
 '/': 7,
 '0': 17,
 '1': 3,
 '2': 19,
 '3': 14,
 '4': 18,
 '5': 16,
 '6': 15,
 '7': 11,
 '8': 20,
 '9': 22,
 '<end>': 30,
 '<start>': 29,
 '^': 21,
 'n0': 4,
 'n1': 2,
 'n10': 25,
 'n11': 26,
 'n13': 27,
 'n15': 28,
 'n2': 6,
 'n3': 9,
 'n4': 8,
 'n5': 12,
 'n6': 10,
 'n7': 13,
 'n8': 23,
 'n9': 24}

In [ ]:
output_vocab_only_n={'n0':4,'n1':2,'n2':6,'n3':9, 'n4': 8,'n5': 12,'n6': 10,'n7': 13,'n8': 23,'n9': 24,'n10': 25,'n11': 26,'n13': 27,'n15': 28}
output_vocab_only_operators={'*':0,'+':5,'-':1,'/':7,'^':21}
output_vocab_only_constants={'0':17,'1':3,'2':19,'3':14,'4':18,'5':16,'6':15,'7':11,'8':20,'9':22}
word_to_index_output_loss=dict()
idx=0
for operator in output_vocab_only_operators.keys():
  word_to_index_output_loss[operator]=idx
  idx+=1

for operator in output_vocab_only_constants.keys():
  word_to_index_output_loss[operator]=idx
  idx+=1

for operator in output_vocab_only_n.keys():
  word_to_index_output_loss[operator]=idx
  idx+=1

print(word_to_index_output_loss)

{'*': 0, '+': 1, '-': 2, '/': 3, '^': 4, '0': 5, '1': 6, '2': 7, '3': 8, '4': 9, '5': 10, '6': 11, '7': 12, '8': 13, '9': 14, 'n0': 15, 'n1': 16, 'n2': 17, 'n3': 18, 'n4': 19, 'n5': 20, 'n6': 21, 'n7': 22, 'n8': 23, 'n9': 24, 'n10': 25, 'n11': 26, 'n13': 27, 'n15': 28}


<h1> Decoder </h1>

In [ ]:
print(torch.tensor(prepared_questions).shape)

torch.Size([22485, 135])


In [ ]:
# q=torch.tensor([[[1,2,3,9,10,11,12,14]],[[4,5,6,90,80,34,67,54]]])
# rep=torch.tensor([[[7,8,10,11],[17,18,110,111],[71,81,101,111]],[[72,82,102,112],[217,218,2110,2111],[731,831,1301,1311]]])
# print(q.shape)
# print(rep.shape)
# print(torch.tile(q,(1,3,1)))
# q=torch.tile(q,(1,3,1))
# print(torch.tile(q,(1,3,1)).shape)
# print(torch.cat((q,rep),2))

In [ ]:
class Attention_Block(torch.nn.Module):
  def __init__(self,hidden_dim):
    super().__init__()
    self.hidden_dim=hidden_dim
    self.linear_1=nn.Linear(2*hidden_dim,hidden_dim,bias=False)
    self.tanh=nn.Tanh()
    self.linear_2=nn.Linear(hidden_dim,1,bias=False)
    self.softmax=nn.Softmax(dim=1)

  def forward(self,goal,all_rep):
    #Convert Goal into required dimension (n*w*2d) than concat is possible
    # May be deep copy (n*d), (n*w*d) ,(n*w*d) 
    goal=torch.tile(torch.reshape(goal,(goal.shape[0],1,goal.shape[1])),(1,max_len,1))
    #print(goal.shape)
    #print(all_rep)
    input_vector=torch.cat((goal,all_rep),2)
    #print(input_vector.shape)
    x=self.linear_1(input_vector)
    x=self.tanh(x)
    scores=self.linear_2(x)
    #print(scores)
    #print(scores.shape)
    weights=self.softmax(scores)
    #print(weights)
    #print(weights)
    
    #
    ctx_vector=torch.einsum('nwd,nwe->nd',all_rep,weights)
    # print(ctx_vector)
    # print(ctx_vector.shape)

    return ctx_vector





In [ ]:
# test_attention=Attention_Block(4)
# q=torch.tensor([[1.0,2.0,3.0,9.0],[4.0,5.0,6.0,90.0]])
# rep=torch.tensor([[[7.0,8.0,10.0,11.0],[17.0,18.0,110.0,111.0],[71.0,81.0,101.0,111.0]],[[72.0,82.0,102.0,112.0],[217.0,218.0,2110.0,2111.0],[731.0,831.0,1301.0,1311.0]]])
# print(q.shape)
# print(rep.shape)
# test_attention(q,rep).shape

In [ ]:
# weights=[[[1],[2],[3]],[[5],[6],[7]]]
# all_rep=[[[5,6],[7,8],[9,10]],[[12,13],[15,16],[1,1]]]
# print(torch.tensor(all_rep))
# print(torch.tensor(weights))
# torch.einsum('nwd,nwe->nd',torch.tensor(all_rep),torch.tensor(weights))

In [ ]:
class Test(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.embedding_operator=nn.Embedding(output_vocab_size,4,padding_idx=0)
    self.softmax=nn.Softmax()
  def forward(self,x):
    return self.softmax(self.embedding_operator(x))

In [ ]:
test=Test()
test(torch.tensor([0]))[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


tensor([0.2500, 0.2500, 0.2500, 0.2500], grad_fn=<SelectBackward>)

In [ ]:
class Decoder_GT(torch.nn.Module):
  def __init__(self,hidden_dim):
    super().__init__()
    self.embedding_operator=nn.Embedding(output_vocab_size,hidden_dim)
    self.embedding_constants=nn.Embedding(output_vocab_size,hidden_dim)
    self.linear=nn.Linear(hidden_dim+2*hidden_dim,hidden_dim,bias=False)
    self.linear_1=nn.Linear(hidden_dim,1,bias=False)
    self.linear_sig=nn.Linear(3*hidden_dim,hidden_dim,bias=False)
    self.linear_sig_right=nn.Linear(3*hidden_dim,hidden_dim,bias=False)
    self.sigmoid=nn.Sigmoid()
    self.linear_tanh=nn.Linear(3*hidden_dim,hidden_dim,bias=False)
    self.linear_sig_child=nn.Linear(hidden_dim,hidden_dim,bias=False)
    self.linear_tanh_child=nn.Linear(hidden_dim,hidden_dim,bias=False)
    self.linear_tanh_right=nn.Linear(3*hidden_dim,hidden_dim,bias=False)
    self.linear_sig_child_right=nn.Linear(hidden_dim,hidden_dim,bias=False)
    self.linear_tanh_child_right=nn.Linear(hidden_dim,hidden_dim,bias=False)
    self.tanh=nn.Tanh()
    self.hidden_dim=hidden_dim
    self.attention=Attention_Block(hidden_dim)
    self.softmax=nn.Softmax()

  def forward(self,goal_parent,ctx_vector_parent,all_rep,unknowns_list_dict,output_vocab_only_operators,output_vocab_only_constants,token_embedding_parent,left_child=True,target=0,is_train=True,is_root=False,idx=0):
    #Send The parent Operator here
    #Dimension goal,ctx_vector n*d
    #print(goal_parent)
    if (is_root):
      goal=goal_parent
    else:
      if left_child:

        concat_interim=torch.cat((goal_parent,ctx_vector_parent),1)
        final_cat=torch.cat((concat_interim,token_embedding_parent),1)
        sig_out=self.sigmoid(self.linear_sig(final_cat))
        tan_out=self.tanh(self.linear_tanh(final_cat))
        sig_tan=torch.mul(sig_out,tan_out)
        g_l=self.sigmoid(self.linear_sig_child(sig_tan))
        q_le=self.tanh(self.linear_tanh_child(sig_tan))
        goal=torch.mul(g_l,q_le)
      
      else:

        concat_interim=torch.cat((goal_parent,ctx_vector_parent),1)
        final_cat=torch.cat((concat_interim,token_embedding_parent),1)
        sig_out=self.sigmoid(self.linear_sig_right(final_cat))
        tan_out=self.tanh(self.linear_tanh_right(final_cat))
        sig_tan=torch.mul(sig_out,tan_out)
        g_l=self.sigmoid(self.linear_sig_child_right(sig_tan))
        q_le=self.tanh(self.linear_tanh_child_right(sig_tan))
        goal=torch.mul(g_l,q_le)

      #print(goal)

      # goal=calculat_root_goal()

    ctx_vector=self.attention(goal,all_rep)
    #print(ctx_vector)
    if is_root:

      with torch.no_grad():
        for elem in unknowns_list_dict.keys():
          if unknowns_list_dict[elem] != -1:
            unknowns_list_dict[elem]=copy.deepcopy(all_rep[0][unknowns_list_dict[elem]])
            
          else:
            unknowns_list_dict[elem]=torch.tensor([0 for i in range(self.hidden_dim)])
        #print(unknowns_list_dict)

    inp_list=list() #MAY BE PROBLEMATIC IN BACKPROP

    for operator in output_vocab_only_operators.keys():
      operator_emb=self.embedding_operator(torch.tensor([output_vocab_only_operators[operator]]))
      concat_goal_ctx=torch.cat((goal,ctx_vector),1)
      # print(concat_goal_ctx.shape)
      # print(operator_emb.shape)
      final_concat=torch.cat((concat_goal_ctx,operator_emb),1)
      interim=self.linear(final_concat)
      interim=self.tanh(interim)
      interim=self.linear_1(interim)
      inp_list.append(interim)
    
    for constant in output_vocab_only_constants.keys():
      constant_emb=self.embedding_constants(torch.tensor([output_vocab_only_constants[constant]]))
      # print(constant_emb)
      # harsh_exit()
      concat_goal_ctx=torch.cat((goal,ctx_vector),1)
      final_concat=torch.cat((concat_goal_ctx,constant_emb),1)
      interim=self.linear(final_concat)
      interim=self.tanh(interim)
      interim=self.linear_1(interim)
      inp_list.append(interim)
    
    for unknown_embedding in unknowns_list_dict.values():
      concat_goal_ctx=torch.cat((goal,ctx_vector),1)
      #print(unknown_embedding.shape)
      final_concat=torch.cat((concat_goal_ctx,torch.unsqueeze(unknown_embedding, 0)),1)
      interim=self.linear(final_concat)
      interim=self.tanh(interim)
      interim=self.linear_1(interim)
      inp_list.append(interim)
    #print(inp_list)
    #May be problematic in backprop check this first
    out_tensor=inp_list[0]
    
    for k in range(len(inp_list)):
      if k!=0:
        out_tensor=torch.cat((out_tensor,inp_list[k]),1)
    #print(out_tensor)
    prob_y=self.softmax(out_tensor)
    # print(idx)
    #print(prob_y)
    #print(prob_y.shape)
    # print(target[idx])
    if is_train:

      if target[idx] in output_vocab_only_operators.keys():
        ret_list=prob_y
        left_append,l_idx=self.forward(goal,ctx_vector,all_rep,unknowns_list_dict,output_vocab_only_operators,output_vocab_only_constants,self.embedding_operator(torch.tensor([output_vocab_only_operators[target[idx]]])),True,target,True,False,idx+1)
        ret_list=torch.cat((ret_list,left_append),0)
        
        right_append,r_idx=self.forward(goal,ctx_vector,all_rep,unknowns_list_dict,output_vocab_only_operators,output_vocab_only_constants,self.embedding_operator(torch.tensor([output_vocab_only_operators[target[idx]]])),False,target,True,False,l_idx+1)       
        ret_list=torch.cat((ret_list,right_append),0)
        
        return ret_list,r_idx
      else:
        return prob_y,idx



In [ ]:
# encoder_model=Encoder_GT(input_vocab_size,5,4)
# question=[[1,25,3]]
# unknowns=[{'n0':1,'n1':-1,'n2':-1,'n3':-1}]
# goal,gru_rep=encoder_model(torch.tensor(question))
# # print(gru_rep)
# # print(gru_rep[0][1])

In [ ]:

# ctx_vector=torch.tensor([[0,0,0,0]])

# decoder=Decoder_GT(4)
# decoder(goal,ctx_vector,gru_rep,unknowns[0],output_vocab_only_operators,output_vocab_only_constants,0,False,["*","+","n0","*","n1","n2","n3"],True,True,0)
# #(goal,ctx_vector,all_rep,unknowns_list_dict,output_vocab_only_operators,output_vocab_only_constants,self.embedding_operator(torch.tensor([output_vocab_only_operators[target[idx]]])),True,target,True,False,idx+1)

<h1> Making Encoder-Decoder </h1>

In [ ]:
class encoder_decoder_GT(torch.nn.Module):
  def __init__(self,input_vocab_size,embedding_dim,hidden_dim):
    super().__init__()
    self.encoder=Encoder_GT(input_vocab_size,embedding_dim,hidden_dim)
    self.decoder=Decoder_GT(hidden_dim)
  
  def forward(self,questions,target,unknowns_dict):
    #decoder(goal,ctx_vector,gru_rep,unknowns[0],output_vocab_only_operators,output_vocab_only_constants,0,False,["*","+","n0","*","n1","n2","n3"],True,True,0)
    root_goal,all_rep=self.encoder(questions)
    #print(root_goal.shape)
    #print(all_rep.shape)
    #ctx_vect
    return self.decoder(root_goal,0,all_rep,unknowns_dict,output_vocab_only_operators,output_vocab_only_constants,0,False,target,True,True,0)
    




<h1>Making Data Loader</h1>

In [ ]:
class Math_Dataset(Dataset):
  def __init__(self,Q,E,U):
    #super().__init__()
    self.Q=Q
    self.E=E
    self.U=U

  def __len__(self):
    return len(self.Q)

  def __getitem__(self,idx):
    print(self.E[idx])
    return torch.tensor(self.Q[idx]),self.E[idx],self.U[idx]

In [ ]:
train_ds=Math_Dataset(prepared_questions,prefix_equations_list,unknowns_dict_list)
train_dl=DataLoader(train_ds,batch_size=1)




<h1> Training The Model </h1>


In [ ]:
def train(model,epochs=1000,lr=0.001):
  # model.to(device)
  parameters=filter(lambda p: p.requires_grad, model.parameters())
  optimizer = torch.optim.Adam(parameters, lr=lr)
  loss = nn.NLLLoss()


  for epoch in range(0,epochs):
    correct=0
    total=0
    sum_loss=0
    idx_cnt=0
    total_elem=0
    model.train()
    for idx in range(len(prepared_questions)):
      q=torch.tensor([prepared_questions[idx]])
      t=prefix_equations_list[idx]
      u=copy.deepcopy(unknowns_dict_list[idx])
      #print(idx)
      e=list()
      for elem in t:
        e.append(word_to_index_output_loss[elem])
      e=torch.tensor([e])
      #predicted_ans,_=model(q,t,u)
      #print(len(predicted_ans))
      #print(len(predicted_ans[0]))
      #print(predicted_ans)
      # print(t)
      # print(predicted_ans.shape)
      # print(e[0].shape)
      # print(e[0])
      
      if True:
        #print(u)
        predicted_ans,_=model(q,t,u)
        optimizer.zero_grad()
        if e[0].shape[0]==predicted_ans.shape[0]:
          #print(predicted_ans.shape)
          #print(e[0])
          #print(predicted_ans)
          
          #print(predicted_ans.unsqueeze(0).shape)
          #print(e[0].shape)
          #print(e.shape)
          output=0
          
          output+=loss(predicted_ans.unsqueeze(0)[0],e[0])
          #print(output)
          output.backward()
          optimizer.step()
          pred=torch.max(predicted_ans, 1)[1]
          #print(pred)
          #print(e[0])
          for j in range(len(e[0])):
            if e[0][j]==pred[j]:
              correct+=1
            total+=1
        else:
          idx_cnt+=1
        #harsh_exit344()
      # print(output)
      # harsh_exit()
      
      if idx%1000==0:
        print("Train Accuracy=",(correct/(total)), "data processed=",idx)
        print(pred)
        #print(e[0])

    if(epoch%1==0):
      print(epoch,"th Epoch, Train Accuracy=",(correct/total),end=' ')



<h1> Making the Model </h1>

In [ ]:
model=encoder_decoder_GT(input_vocab_size,200,250)

train(model)

In [ ]:
def save_check_point(model,valid_loss=0):
  #model.to(device)
  #valid_loss.to(device)
  state_dict={'model_state_dict':model.state_dict(),'valid_loss':valid_loss}
  torch.save(state_dict,"/content/drive/MyDrive/model_200d.pth")
  print("Model Check Point Saved Succesfully")

In [ ]:
print(prepared_questions)

In [ ]:
save_check_point(model)

In [ ]:
print(prepared_questions[4403])
print(prefix_equations_list[4403])

In [ ]:
idx=0
for name,_ in  model.named_parameters():
  idx+=1
  print(name)
print(idx)


In [ ]:
idx=0
for name,param in  model.named_parameters():
  if param.require_grad==True:
    idx+=1
    print(name)
print(idx)


In [ ]:
for i in  range(len(list(model.named_parameters()))):
  print(list(model.named_parameters())[i])
  print(list(model.parameters())[i])
  